In [2]:
# !pip install pdfplumber

   ---------------------------------------- 5.6/5.6 MB 15.6 MB/s eta 0:00:00
   ---------------------------------------- 3.0/3.0 MB 15.7 MB/s eta 0:00:00


In [3]:
import nltk
import re
import pdfplumber

In [15]:
# Download NLTK stopwords (required for preprocessing)
nltk.download('stopwords')
from nltk.corpus import stopwords
# Download the 'averaged_perceptron_tagger' resource
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [5]:
# Define skills and education keywords
SKILLS = [
    "Python", "Java", "C++", "SQL", "Machine Learning", "Data Analysis",
    "Django", "Flask", "HTML", "CSS", "React", "Node.js", "Agile", "Git"
]
EDUCATION_KEYWORDS = [
    "Bachelor", "Master", "B.Tech", "M.Tech", "B.Sc", "M.Sc", "PhD", 
    "High School", "Secondary School", "Diploma"
]


In [6]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [7]:
# Function to preprocess text
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words and word.isalnum()]
    return " ".join(filtered_words)

In [8]:
# Function to extract email
def extract_email(text):
    email_pattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
    email = re.findall(email_pattern, text)
    return email[0] if email else None

In [9]:
# Function to extract phone number
def extract_phone_number(text):
    phone_pattern = r'\b\d{10}\b|\+?\d{1,4}[-.\s]?\(?\d{1,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,9}'
    phone = re.findall(phone_pattern, text)
    return phone[0] if phone else None

In [10]:
# Function to extract skills
def extract_skills(text):
    skills_found = [skill for skill in SKILLS if skill.lower() in text.lower()]
    return skills_found

In [11]:
# Function to extract education details
def extract_education(text):
    education_lines = []
    for line in text.split('\n'):
        if any(keyword in line for keyword in EDUCATION_KEYWORDS):
            education_lines.append(line.strip())
    return education_lines

In [12]:
# Function to extract name
def extract_name(text):
    tokens = nltk.word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)
    for i, (word, tag) in enumerate(tagged_tokens):
        if tag == "NNP":  # Proper noun
            return word
    return None

In [18]:
# Main function to parse resume
def parse_resume(pdf_path):
    raw_text = extract_text_from_pdf(pdf_path)
    clean_text = preprocess_text(raw_text)

    name = extract_name(clean_text)
    email = extract_email(raw_text)
    phone = extract_phone_number(raw_text)
    skills = extract_skills(clean_text)
    #education = extract_education(raw_text)

    return {
        "Name": name,
        "Email": email,
        "Phone": phone,
        "Skills": skills,
        #"Education": education,
    }

In [ ]:
# Example usage
if __name__ == "__main__":
    pdf_path = "Rahul_K_Resume.pdf"  # Replace with the path to your resume
    parsed_data = parse_resume(pdf_path)
    for key, value in parsed_data.items():
        print(f"{key}: {value}")